<a href="https://colab.research.google.com/github/pairuoyefe/ChatBot/blob/main/Chatbot2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import numpy as np
import random
import re

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.preprocessing import LabelEncoder


In [5]:
# Load JSON
with open("/content/starwarsintents.json") as file:
    data = json.load(file)

# Show example
print(data)



{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'Hey', 'How are you', 'Is anyone there?', 'Hello', 'Good day', "What's up", 'Yo!', 'Howdy', 'Nice to meet you.'], 'responses': ['Hey', 'Hello, thanks for visiting.', 'Hi there, what can I do for you?', 'Hi there, how can I help?', 'Hello, there.', 'Hello Dear', 'Ooooo Hello, looking for someone or something?', 'Yes, I am here.', 'Listening carefully.', 'Ok, I am with you.']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later.', 'Goodbye', 'Have a great day.', 'See you next time.', 'It was my pleassure.', 'Take care.', 'See ya!', 'Catch you later.', 'Ciao.'], 'responses': ['See you later, thanks for visiting.', 'May the force be with you!', 'See next time.', 'Was my pleassuare to meet you.', 'Hope will cath up sortly.', 'Have a nice day.', 'Bye! Come back again soon.', 'So, till next time.', 'If you need anything just text me anytime. Bye.', 'Well, hope see you soon!']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's

In [8]:
# Collect training data
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        texts.append(pattern)
        labels.append(intent["tag"])

# Encode labels
lbl_encoder = LabelEncoder()
labels_encoded = lbl_encoder.fit_transform(labels)

# Tokenize text
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded_sequences = pad_sequences(sequences, padding="post")


In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

vocab_size = len(tokenizer.word_index) + 1
output_size = len(lbl_encoder.classes_)

model = Sequential()
model.add(Dense(64, input_shape=(padded_sequences.shape[1],), activation='relu'))
model.add(Dense(output_size, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.summary()



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_7 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 16)             │         1,040 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,872 (7.31 KB)

 Trainable params: 1,872 (7.31 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
model.fit(padded_sequences, labels_encoded, epochs=50, batch_size=64, validation_split=0.2)


Epoch 1/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 378ms/step - accuracy: 0.0207 - loss: 26.9041 - val_accuracy: 0.0513 - val_loss: 17.7088
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.0227 - loss: 24.5360 - val_accuracy: 0.0769 - val_loss: 15.6625
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.0414 - loss: 22.9108 - val_accuracy: 0.0769 - val_loss: 13.8953
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.0765 - loss: 19.7063 - val_accuracy: 0.1282 - val_loss: 12.4075
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.1031 - loss: 17.0188 - val_accuracy: 0.1282 - val_loss: 11.1152
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.1205 - loss: 15.6466 - val_accuracy: 0.1538 - val_loss: 10.0896
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.1789 - loss: 14.0854 - val_accuracy: 0.1795 - val_loss: 9.3909
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.2807 - loss: 12.7891 - val_accuracy: 0.1795 - va

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(texts)  # 'texts' is your list of patterns


In [21]:
from sklearn.preprocessing import LabelEncoder

lbl_encoder = LabelEncoder()
y = lbl_encoder.fit_transform(labels)


In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier  # or KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from  sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

# Initialize TF-IDF vectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(texts)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize model (choose one)
model = RandomForestClassifier()  # or KNeighborsClassifier(n_neighbors=3)

# Train
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)

# Check accuracy score
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.7948717948717948


In [32]:
import random

def chatbot_response(Hi):
    # Convert user input to TF-IDF vector (same vectorizer used in training)
    X_input = vectorizer.transform([user_input])

    # Predict intent label index
    pred = model.predict(X_input)

    # Convert label index back to tag
    tag = lbl_encoder.inverse_transform(pred)[0]

    # Find matching responses from your intents data (make sure 'data' is loaded)
    for intent in data["intents"]:
        if intent["tag"] == tag:
            return random.choice(intent["responses"])

    return "Sorry, I don't understand."


In [33]:
import pickle

# Save the trained model
with open("rf_chatbot_model.pkl", "wb") as f:
    pickle.dump(model, f)

# Save the TF-IDF vectorizer
with open("tfidf_vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)

# Save the label encoder
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(lbl_encoder, f)
